In [16]:
import pandas as pd

In [17]:
df = pd.read_csv("results_full.csv")
df = df[df["split_method"] == "random_split"]

In [18]:
rename_map = {
    'ConstantPredictor': 'Const.',
    'FTTransformer': 'FT-Trans.',
    'LGBMRegressor': 'GBT',
    'LGBMClassifier': 'GBT',
    'LinearRegressor': 'Lin. Regr.',
    'RandomForestRegressor': 'RF',
    'RandomForestClassifier': 'RF',
    'ResNet': 'ResNet',
    'TabPFNRegressor': 'TabPFN',
    'TabPFNClassifier': 'TabPFN',
    'LogisticRegressor': 'Log. Regr.',
    'GPBoost_LogLoss': 'GP',
    'GPBoost_Accuracy': 'GP',
    'GPBoost_CRPS': 'GP',
    'GPBoost_RMSE': 'GP'

}


Average Difference and Average Relative Difference RMSE

In [5]:
metric = "RMSE"
dfm   = df[df["metric"] == metric]

pivot_rmse = dfm.pivot_table(
    index="task_id",
    columns="model",
    values="value",
    aggfunc="mean"
)
pivot_rmse = pivot_rmse.rename(columns=rename_map)

pivot_rmse.to_csv(f"avg_{metric}_per_task_per_model_IP.csv", float_format="%.3f")


In [6]:
def avg_relative_diff(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:

    if error_metric:
        best_per_task = pivot.min(axis=1)   # best is minimum
        # (v - best)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)
                         .div(best_per_task, axis=0)
                         * 100)
    else:
        best_per_task = pivot.max(axis=1)   # best is maximum
        # (best - v)/best * 100
        rel_diff = (best_per_task.sub(pivot, axis=0)
                              .div(best_per_task, axis=0)
                              * 100)

    # average across tasks (skipping any NaNs)
    return rel_diff.mean(axis=0)


avg_diff_rmse = avg_relative_diff(pivot_rmse, error_metric=True)

print("Average relative difference (RMSE) - Interpolation :")
print(avg_diff_rmse.round(2))


Average relative difference (RMSE) - Interpolation :
model
Const.          357.98
Engression      475.69
FT-Trans.        32.15
GP              197.49
GBT              66.13
Lin. Regr.    79680.52
MLP             110.19
RF               69.38
ResNet          100.48
TabPFN           30.79
dtype: float64


In [7]:
def normalized_accuracy(pivot: pd.DataFrame,
                        error_metric: bool = True) -> pd.Series:

    if error_metric:
        best = pivot.min(axis=1)  

        mid  = pivot.apply(
            lambda s: s.dropna().nlargest(3).min(),
            axis=1
        )
        norm = (
            pivot
            .rsub(mid,       axis=0)    # mid[t] - pivot.loc[t, m]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )
    else:
        best = pivot.max(axis=1)
        mid  = pivot.apply(
            lambda s: s.dropna().nsmallest(3).max(),
            axis=1
        )
        # 3) norm_acc per cell = (acc - mid) / (best - mid), clipped
        norm = (
            pivot
            .rsub(mid,          axis=0)   # pivot.loc[t,m] - mid[t]
            .mul(-1)
            .div(mid - best, axis=0)
            .clip(0, 1)
        )

    return norm.mean(axis=0)


avg_norm_acc_rmse = normalized_accuracy(pivot_rmse, error_metric=True)
print("Average normalized accuracy (from RMSE) - Interpolation:")
print((100 * avg_norm_acc_rmse).round(2))


Average normalized accuracy (from RMSE) - Interpolation:
model
Const.         8.98
Engression    36.13
FT-Trans.     64.98
GP            31.89
GBT           76.27
Lin. Regr.     6.75
MLP           56.22
RF            69.79
ResNet        45.87
TabPFN        89.82
dtype: float64


In [8]:
def avg_rank(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:
  
    ranks = pivot.rank(
        axis=1,
        method="average",   
        ascending=error_metric
    )

    return ranks.mean(axis=0)

avg_rank_rmse = avg_rank(pivot_rmse, error_metric=True)
print("Average rank (RMSE) - Interpolation:")
print(avg_rank_rmse.round(2))



Average rank (RMSE) - Interpolation:
model
Const.        8.92
Engression    6.28
FT-Trans.     4.09
GP            6.53
GBT           3.25
Lin. Regr.    8.42
MLP           5.06
RF            4.19
ResNet        5.75
TabPFN        2.03
dtype: float64


In [9]:
avg_diff    = avg_relative_diff(pivot_rmse, error_metric=True)           # in %
avg_acc     = normalized_accuracy(pivot_rmse, error_metric=True) * 100   # convert to % 
avg_rank    = avg_rank(pivot_rmse, error_metric=True)

metrics = pd.DataFrame(
    [avg_diff, avg_acc, avg_rank],
    index=["Avg. diff.", "Avg. acc.", "Avg. rank."]
)

pivot_with_summaries = pd.concat([pivot_rmse, metrics])

out = pivot_with_summaries.reset_index().rename(columns={"index":"task_id"})

out.to_csv(f"avg_{metric}_with_summary_IP.csv", float_format="%.3f", index=False)

latex = out.to_latex(index=False, escape=True, float_format="%.3f")
with open(f"{metric}_IP_results.tex","w") as f:
    f.write(latex)

Average Difference and Average Relative Difference CRPS

In [10]:
metric1 = "CRPS"
dfm1   = df[df["metric"] == metric1]

pivot_crps = dfm1.pivot_table(
    index="task_id",
    columns="model",
    values="value",
    aggfunc="mean"
)

pivot_crps = pivot_crps.rename(columns=rename_map)
pivot_crps.to_csv(f"avg_{metric1}_per_task_per_model_IP.csv", float_format="%.3f")


In [11]:
def avg_relative_diff(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:

    if error_metric:
        best_per_task = pivot.min(axis=1)   # best is minimum
        # (v - best)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)
                         .div(best_per_task, axis=0)
                         * 100)
    else:
        best_per_task = pivot.max(axis=1)   # best is maximum
        # (best - v)/best * 100
        rel_diff = (best_per_task.sub(pivot, axis=0)
                              .div(best_per_task, axis=0)
                              * 100)

    # average across tasks (skipping any NaNs)
    return rel_diff.mean(axis=0)


avg_diff_crps = avg_relative_diff(pivot_crps, error_metric=True)

print("Average relative difference (CRPS) - Interpolation:")
print(avg_diff_crps.round(2))


Average relative difference (CRPS) - Interpolation:
model
Const.          582.33
DGBT          55968.97
DRF              82.39
Engression      470.83
FT-Trans.        38.26
GP              141.93
GBT              62.19
Lin. Regr.    55235.36
MLP              90.12
RF               73.95
ResNet          119.18
TabPFN           24.38
dtype: float64


In [12]:
def normalized_accuracy(pivot: pd.DataFrame,
                        error_metric: bool = True) -> pd.Series:

    if error_metric:
        best = pivot.min(axis=1)  

        mid  = pivot.apply(
            lambda s: s.dropna().nlargest(3).min(),
            axis=1
        )
        norm = (
            pivot
            .rsub(mid,       axis=0)    # mid[t] - pivot.loc[t, m]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )
    else:
        best = pivot.max(axis=1)
        mid  = pivot.apply(
            lambda s: s.dropna().nsmallest(3).max(),
            axis=1
        )
        # 3) norm_acc per cell = (acc - mid) / (best - mid), clipped
        norm = (
            pivot
            .rsub(mid,          axis=0)   # pivot.loc[t,m] - mid[t]
            .mul(-1)
            .div(mid - best, axis=0)
            .clip(0, 1)
        )

    return norm.mean(axis=0)


avg_norm_acc_crps = normalized_accuracy(pivot_crps, error_metric=True)
print("Average normalized accuracy (from CRPS) - Interpolation:")
print((100 * avg_norm_acc_crps).round(2))


Average normalized accuracy (from CRPS) - Interpolation:
model
Const.         4.68
DGBT          69.39
DRF           75.29
Engression    15.90
FT-Trans.     66.22
GP            41.13
GBT           73.50
Lin. Regr.    11.00
MLP           59.83
RF            68.98
ResNet        52.25
TabPFN        84.29
dtype: float64


In [13]:
def avg_rank(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:
  
    ranks = pivot.rank(
        axis=1,
        method="average",   
        ascending=error_metric
    )

    return ranks.mean(axis=0)

avg_rank_crps = avg_rank(pivot_crps, error_metric=True)
print("Average rank (CRPS) - Interpolation:")
print(avg_rank_crps.round(2))



Average rank (CRPS) - Interpolation:
model
Const.        10.92
DGBT           4.39
DRF            4.47
Engression     9.89
FT-Trans.      4.94
GP             8.03
GBT            4.19
Lin. Regr.     9.86
MLP            6.08
RF             5.25
ResNet         6.69
TabPFN         2.69
dtype: float64


In [14]:
avg_diff1    = avg_relative_diff(pivot_crps, error_metric=True)           # in %
avg_acc1     = normalized_accuracy(pivot_crps, error_metric=True) * 100   # convert to % 
avg_rank1    = avg_rank(pivot_crps, error_metric=True)

metrics = pd.DataFrame(
    [avg_diff1, avg_acc1, avg_rank1],
    index=["Avg. diff.", "Avg. acc.", "Avg. rank."]
)

pivot_with_summaries1 = pd.concat([pivot_crps, metrics])

out1 = pivot_with_summaries1.reset_index().rename(columns={"index":"task_id"})

out1.to_csv(f"avg_{metric1}_with_summary_IP.csv", float_format="%.3f", index=False)

latex1 = out1.to_latex(index=False, escape=True, float_format="%.3f")
with open(f"{metric1}_IP_results.tex","w") as f:
    f.write(latex1)

Average Difference and Average Relative Difference LOGLOSS

In [19]:
metric2 = "LogLoss"
dfm2   = df[df["metric"] == metric2]

pivot_ll = dfm2.pivot_table(
    index="task_id",
    columns="model",
    values="value",
    aggfunc="mean"
)
pivot_ll = pivot_ll.rename(columns=rename_map)
pivot_ll.to_csv(f"avg_{metric2}_per_task_per_model_IP.csv", float_format="%.3f")


In [20]:
def avg_relative_diff(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:

    if error_metric:
        best_per_task = pivot.min(axis=1)   # best is minimum
        # (v - best)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)
                         .div(best_per_task, axis=0)
                         * 100)
    else:
        best_per_task = pivot.max(axis=1)   # best is maximum
        # (best - v)/best * 100
        rel_diff = (best_per_task.sub(pivot, axis=0)
                              .div(best_per_task, axis=0)
                              * 100)

    return rel_diff.mean(axis=0)


avg_diff_ll = avg_relative_diff(pivot_ll, error_metric=True)

print("Average relative difference (LogLoss) - Interpolation:")
print(avg_diff_ll.round(2))


Average relative difference (LogLoss) - Interpolation:
model
Const.        187.13
Engression     22.86
FT-Trans.      15.21
GP             37.21
GBT             4.68
Log. Regr.     85.34
MLP            12.61
RF             14.96
ResNet         17.97
TabPFN          1.19
dtype: float64


In [21]:
def normalized_accuracy(pivot: pd.DataFrame,
                        error_metric: bool = True) -> pd.Series:

    if error_metric:
        best = pivot.min(axis=1)  

        mid  = pivot.apply(
            lambda s: s.dropna().nlargest(3).min(),
            axis=1
        )
        norm = (
            pivot
            .rsub(mid,       axis=0)    # mid[t] - pivot.loc[t, m]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )
    else:
        best = pivot.max(axis=1)
        mid  = pivot.apply(
            lambda s: s.dropna().nsmallest(3).max(),
            axis=1
        )
        # 3) norm_acc per cell = (acc - mid) / (best - mid), clipped
        norm = (
            pivot
            .rsub(mid,          axis=0)   # pivot.loc[t,m] - mid[t]
            .mul(-1)
            .div(mid - best, axis=0)
            .clip(0, 1)
        )

    return norm.mean(axis=0)


avg_norm_acc_ll = normalized_accuracy(pivot_ll, error_metric=True)
print("Average normalized accuracy (from LogLoss) - Interpolation:")
print((100 * avg_norm_acc_ll).round(2))


Average normalized accuracy (from LogLoss) - Interpolation:
model
Const.         0.00
Engression    19.88
FT-Trans.     46.80
GP            29.78
GBT           78.43
Log. Regr.     2.80
MLP           48.11
RF            64.38
ResNet        35.56
TabPFN        96.47
dtype: float64


In [22]:
def avg_rank(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:
  
    ranks = pivot.rank(
        axis=1,
        method="average",   
        ascending=error_metric
    )

    return ranks.mean(axis=0)

avg_rank_ll = avg_rank(pivot_ll, error_metric=True)
print("Average rank (LogLoss) - Interpolation:")
print(avg_rank_ll.round(2))



Average rank (LogLoss) - Interpolation:
model
Const.        9.74
Engression    6.91
FT-Trans.     5.40
GP            5.87
GBT           2.61
Log. Regr.    8.26
MLP           4.65
RF            3.83
ResNet        5.74
TabPFN        1.39
dtype: float64


In [23]:
avg_diff2    = avg_relative_diff(pivot_ll, error_metric=True)           # in %
avg_acc2     = normalized_accuracy(pivot_ll, error_metric=True) * 100   # convert to % 
avg_rank2    = avg_rank(pivot_ll, error_metric=True)

metrics = pd.DataFrame(
    [avg_diff2, avg_acc2, avg_rank2],
    index=["Avg. diff.", "Avg. acc.", "Avg. rank."]
)

pivot_with_summaries2 = pd.concat([pivot_ll, metrics])

out2 = pivot_with_summaries2.reset_index().rename(columns={"index":"task_id"})

out2.to_csv(f"avg_{metric2}_with_summary_IP.csv", float_format="%.3f", index=False)

latex2 = out2.to_latex(index=False, escape=True, float_format="%.3f")
with open(f"{metric2}_IP_results.tex","w") as f:
    f.write(latex2)

Average Difference and Average Relative Difference Accuracy

In [24]:
metric3 = "Accuracy"
dfm3   = df[df["metric"] == metric3]

pivot_acc = dfm3.pivot_table(
    index="task_id",
    columns="model",
    values="value",
    aggfunc="mean"
)

pivot_acc = pivot_acc.rename(columns=rename_map)
pivot_acc.to_csv(f"avg_{metric3}_per_task_per_model_IP.csv", float_format="%.5f")


In [25]:
def avg_relative_diff(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:

    if error_metric:
        best_per_task = pivot.min(axis=1)   # best is minimum
        # (v - best)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)
                         .div(best_per_task, axis=0)
                         * 100)
    else:
        best_per_task = pivot.max(axis=1)   # best is maximum
        # (best - v)/best * 100
        rel_diff = (pivot.sub(best_per_task, axis=0)     
                         .mul(-1)                         
                         .div(best_per_task , axis=0)
                         * 100)

    return rel_diff.mean(axis=0)


avg_diff_accuracy = avg_relative_diff(pivot_acc, error_metric=False)

print("\nAverage relative difference (Accuracy) - Interpolation:")
print(avg_diff_accuracy.round(2))



Average relative difference (Accuracy) - Interpolation:
model
Const.        36.01
Engression     4.60
FT-Trans.      4.48
GP             5.10
GBT            1.07
Log. Regr.     9.29
MLP            3.79
RF             2.71
ResNet         3.33
TabPFN         0.41
dtype: float64


In [26]:
def normalized_accuracy(pivot: pd.DataFrame,
                        error_metric: bool = True) -> pd.Series:

    if error_metric:
        best = pivot.min(axis=1)  

        mid  = pivot.apply(
            lambda s: s.dropna().nlargest(3).min(),
            axis=1
        )
        norm = (
            pivot
            .rsub(mid,       axis=0)    # mid[t] - pivot.loc[t, m]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )
    else:
        best = pivot.max(axis=1)
        mid  = pivot.apply(
            lambda s: s.dropna().nsmallest(3).max(),
            axis=1
        )
        # 3) norm_acc per cell = (acc - mid) / (best - mid), clipped
        norm = (
            pivot
            .rsub(mid,          axis=0)   # pivot.loc[t,m] - mid[t]
            .div(mid - best, axis=0)
            .clip(0, 1)
        )

    return norm.mean(axis=0)


avg_norm_acc_acc = normalized_accuracy(pivot_acc, error_metric=False)
print("Average normalized accuracy (from Accuracy) - Interpolation:")
print((100 * avg_norm_acc_acc).round(3))


Average normalized accuracy (from Accuracy) - Interpolation:
model
Const.         0.000
Engression    30.402
FT-Trans.     44.757
GP            21.548
GBT           79.346
Log. Regr.     7.710
MLP           41.704
RF            52.176
ResNet        51.310
TabPFN        90.492
dtype: float64


In [27]:
def avg_rank(pivot: pd.DataFrame, error_metric: bool = True) -> pd.Series:
  
    ranks = pivot.rank(
        axis=1,
        method="average",   
        ascending=error_metric
    )

    return ranks.mean(axis=0)

avg_rank_acc = avg_rank(pivot_acc, error_metric=False)
print("Average rank (Accuracy) -Interpolation:")
print(avg_rank_acc.round(2))



Average rank (Accuracy) -Interpolation:
model
Const.        9.87
Engression    6.65
FT-Trans.     4.85
GP            6.20
GBT           2.80
Log. Regr.    8.02
MLP           5.24
RF            4.35
ResNet        4.46
TabPFN        1.89
dtype: float64


In [28]:
avg_diff3    = avg_relative_diff(pivot_acc, error_metric=False)           # in %
avg_acc3     = normalized_accuracy(pivot_acc, error_metric=False) * 100   # convert to % 
avg_rank3    = avg_rank(pivot_acc, error_metric=False)

metrics = pd.DataFrame(
    [avg_diff3, avg_acc3, avg_rank3],
    index=["Avg. diff.", "Avg. acc.", "Avg. rank."]
)

pivot_with_summaries3 = pd.concat([pivot_acc, metrics])

out3 = pivot_with_summaries3.reset_index().rename(columns={"index":"task_id"})

out3.to_csv(f"avg_{metric3}_with_summary_IP.csv", float_format="%.3f", index=False)

latex3 = out3.to_latex(index=False, escape=True, float_format="%.3f")
with open(f"{metric3}_IP_results.tex","w") as f:
    f.write(latex3)